This notebook provides a step-by-step guide to run PET.

There are three steps to run PET:
Step 1. Perform differential gene expression analysis using DESeq2
Step 2. Perform pathway analysis using underlying methods (i.e. GSEA, ORA, Enrichr)
Step 3. Combine the outputs from above methods using PET

Required files:
* Pathways to be investigated (e.g. GO biological processes) as [.gmt files](https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#Gene_Set_Database_Formats). You can find an example pathway file [here](https://github.com/hedgehug/PET/blob/main/example/c2.cp.kegg.v2023.1.Hs.symbols.gmt). 
* Expression data from samples to be investigated, as a tab-delimited text file. **Raw counts** are highly recommended as required by DESeq2. The first column of the file should contain the gene symbols and the of rest columns should contain the expression values ordered by conditions. An example expression file is provided [here](https://github.com/hedgehug/PET/tree/main/example/example_data.txt). 
* A design matrix file defining the conditions and number of samples as a tab-delimited text file to be used by DESeq2. The first column should specify the condition name and the second column should indicate the number of samples. **The order and the number of samples should match the expression file above.** An example contrast file can be found [here](https://github.com/hedgehug/PET/tree/main/example/example_contrast.txt). 



Notes:\
We have provided a template script to run GSEA optimally (as defined by Benchmark) and [DESeq2](https://github.com/hedgehug/PET/blob/main/deseq2_template.R). However, these analyses could be performed using any different settings as long as the resulting file format meets PET's required format.


In [1]:
from ora import run_ora
from enrichr import run_enrichr
from PET import run_PET
from helper import *

In [2]:
# create result directory
out_dir = 'example/'
create_dir(out_dir)

**Step 1. Perform differential gene expression analysis using DESeq2**\

Based on Benchmark, using a pre-ranked gene list derived from the p-values of DESeq2 analysis provides an optimal setting for pathway analysis.\

PET runs DESeq2 using either:
* [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) in R, default option. 

* [PyDESeq2](https://pydeseq2.readthedocs.io/en/latest/), when method='Python' specified. 

\
Please note that **discrepancies** have been observed between the R implementation of DESeq2 and PyDESeq2. Therefore, we suggest continuing to use the default DESeq2 version for R.


In [3]:
# To run differential expression analysis using default DESeq2 R version as below:
# note that by default, PET saves all results to the result_dir_path (as defined above)

run_DEseq2(expr_file=out_dir+'example_data.txt', contrast_matrix_file=out_dir+'example_contrast.txt', 
           result_dir_path=out_dir, method='R', script_file_name=out_dir+'deseq_analysis.all.R')


DESeq2 script written to example/deseq_analysis.all.R
Start running R script example/deseq_analysis.all.R


Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The followin

class: DESeqDataSet 
dim: 53657 10 
metadata(1): version
assays(1): counts
rownames(53657): TSPAN6 TNMD ... LINC01144 AC007389.5
rowData names(0):
colnames(10): TCGA.AA.3850 TCGA.AA.3845 ... TCGA.DM.A1HA TCGA.DM.A1HB
colData names(1): condition


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] "Comparing cond1 vs. cond2"
[1] "Result saved to  example//cond1.vs.cond2.csv"
[1] "Comparing cond1 vs. cond3"
[1] "Result saved to  example//cond1.vs.cond3.csv"
[1] "Comparing cond2 vs. cond3"
[1] "Result saved to  example//cond2.vs.cond3.csv"
Analysis Done!


In [4]:
# if only one of the comparisons is needed for the downstream analyses
# please provide group=[condition1, condition2] and result_file_path
run_DEseq2(expr_file=out_dir+'example_data.txt', contrast_matrix_file=out_dir+'example_contrast.txt', 
           groups= ['cond2', 'cond1'], result_file_path=out_dir+'cond2.vs.cond1.deseq_result.txt', 
           method='R', script_file_name=out_dir+'deseq_analysis.cond2.vs.cond1.R')

DESeq2 script written to example/deseq_analysis.cond2.vs.cond1.R
Start running R script example/deseq_analysis.cond2.vs.cond1.R


Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The followin

class: DESeqDataSet 
dim: 53657 10 
metadata(1): version
assays(1): counts
rownames(53657): TSPAN6 TNMD ... LINC01144 AC007389.5
rowData names(0):
colnames(10): TCGA.AA.3850 TCGA.AA.3845 ... TCGA.DM.A1HA TCGA.DM.A1HB
colData names(1): condition


estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] "Comparing cond2 vs. cond1"
[1] "Result saved to  example/cond2.vs.cond1.deseq_result.txt"
Analysis Done!


In [5]:
# Alternatively, to run PyDESeq2, similar to DESeq2 R version, please specify method='Python' as below: 
# By default, PET will save all comparisons results to result_dir_path
run_DEseq2(expr_file=out_dir+'example_data.txt', contrast_matrix_file=out_dir+'example_contrast.txt', 
           result_dir_path=out_dir, method='Python', cpu_num=10)

10 samples
53657 genes
3 conditions


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 4.31 seconds.

Fitting dispersion trend curve...
... done in 2.34 seconds.

Fitting MAP dispersions...
... done in 4.64 seconds.

Fitting LFCs...
... done in 4.34 seconds.

Refitting 0 outliers.

Running Wald tests...


Comparing  cond1 .vs. cond2


... done in 2.33 seconds.



Log2 fold change & Wald test p-value: condition cond1 vs cond2
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Gene                                                                    
TSPAN6      2713.493826       -0.512394  0.456068 -1.123502  0.261224   
TNMD           5.937313       -0.513815  1.363464 -0.376845  0.706288   
DPM1        1127.214535       -0.527339  0.677807 -0.778007  0.436565   
SCYL3        461.262837       -0.107010  0.316047 -0.338590  0.734919   
C1orf112     336.957075       -1.047450  0.456055 -2.296763  0.021632   
...                 ...             ...       ...       ...       ...   
LINC02246      9.395658        0.741933  1.228398  0.603984  0.545854   
AC233263.6     0.221389        1.398720  4.213062  0.331996  0.739892   
HERC2P7        0.053885        0.656822  4.346304  0.151122  0.879880   
LINC01144     23.140440        0.855266  0.713613  1.198500  0.230722   
AC007389.5     0.868972        1.359826  3.215577  0.422887  

Running Wald tests...
... done in 2.33 seconds.



Log2 fold change & Wald test p-value: condition cond1 vs cond3
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Gene                                                                    
TSPAN6      2713.493826        1.154604  0.456367  2.529992  0.011407   
TNMD           5.937313        2.406839  1.487954  1.617549  0.105760   
DPM1        1127.214535       -0.448179  0.677761 -0.661265  0.508442   
SCYL3        461.262837       -0.322971  0.315385 -1.024053  0.305810   
C1orf112     336.957075       -0.042617  0.456985 -0.093257  0.925699   
...                 ...             ...       ...       ...       ...   
LINC02246      9.395658       -2.181930  1.145896 -1.904126  0.056894   
AC233263.6     0.221389        0.656488  3.986285  0.164687  0.869191   
HERC2P7        0.053885       -0.085411  4.126855 -0.020696  0.983488   
LINC01144     23.140440       -0.691821  0.686955 -1.007084  0.313895   
AC007389.5     0.868972        3.662723  3.631056  1.008721  

Running Wald tests...
... done in 2.40 seconds.



Log2 fold change & Wald test p-value: condition cond2 vs cond3
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Gene                                                                    
TSPAN6      2713.493826        1.666998  0.487673  3.418266  0.000630   
TNMD           5.937313        2.920655  1.553067  1.880572  0.060030   
DPM1        1127.214535        0.079159  0.724219  0.109303  0.912962   
SCYL3        461.262837       -0.215961  0.336120 -0.642510  0.520542   
C1orf112     336.957075        1.004832  0.486094  2.067159  0.038719   
...                 ...             ...       ...       ...       ...   
LINC02246      9.395658       -2.923862  1.242310 -2.353570  0.018594   
AC233263.6     0.221389       -0.742232  4.445106 -0.166977  0.867388   
HERC2P7        0.053885       -0.742232  4.445107 -0.166977  0.867388   
LINC01144     23.140440       -1.547086  0.742774 -2.082851  0.037265   
AC007389.5     0.868972        2.302897  3.872216  0.594723  

In [6]:
# similarly, if only one of the comparison is needed for the downstream analyses
# please provide group=[condition1, condition2] and result_file_path
run_DEseq2(expr_file=out_dir+'example_data.txt', contrast_matrix_file=out_dir+'example_contrast.txt', 
           groups= ['cond2', 'cond1'], result_file_path=out_dir+'cond2.vs.cond1.PyDESeq2_result.csv', 
           method='Python', cpu_num=10)

10 samples
53657 genes
3 conditions


Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 4.39 seconds.

Fitting dispersion trend curve...
... done in 2.12 seconds.

Fitting MAP dispersions...
... done in 4.69 seconds.

Fitting LFCs...
... done in 4.33 seconds.

Refitting 0 outliers.

Running Wald tests...


Comparing  cond2 .vs. cond1


... done in 2.41 seconds.



Log2 fold change & Wald test p-value: condition cond2 vs cond1
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Gene                                                                    
TSPAN6      2713.493826        0.512394  0.456068  1.123502  0.261224   
TNMD           5.937313        0.513815  1.363464  0.376845  0.706288   
DPM1        1127.214535        0.527339  0.677807  0.778007  0.436565   
SCYL3        461.262837        0.107010  0.316047  0.338590  0.734919   
C1orf112     336.957075        1.047450  0.456055  2.296763  0.021632   
...                 ...             ...       ...       ...       ...   
LINC02246      9.395658       -0.741933  1.228398 -0.603984  0.545854   
AC233263.6     0.221389       -1.398720  4.213062 -0.331996  0.739892   
HERC2P7        0.053885       -0.656822  4.346304 -0.151122  0.879880   
LINC01144     23.140440       -0.855266  0.713613 -1.198500  0.230722   
AC007389.5     0.868972       -1.359826  3.215577 -0.422887  

Note that a prefix/comparison_name is specified for all the output files and PET will use files associated with this prefix for the following steps.

In [3]:
prefix = 'cond1.vs.cond3'

In [16]:
# To generate rank file (.rnk) for GSEA analysis from DESeq2 result in CSV format:
# Please double check to ensure that the direction reported by DESeq2 result (i.e. log2FoldChange) matches the desired comparison 
# if the direction does not match the desired contrast, you may change the direction=-1.

generate_rank_file(deseq_result_file=out_dir+prefix+'.csv', out_file=out_dir+prefix+'.rnk', direction=1)

Rank file written to  example/cond1.vs.cond3.rnk


**Step 2. Perform pathway analysis using underlying methods (i.e. GSEA, ORA, Enrichr)**

Based on Benchmark, the best practice to run GSEA is providing a pre-ranked gene list ordered based on log10(p-value)*sign(logFC). 

PET can run GSEA using either:

* [GSEApy](https://gseapy.readthedocs.io/en/latest/introduction.html), default option. or
* [GSEA command line (all platforms)](http://www.gsea-msigdb.org/gsea/downloads.jsp).


In [9]:
# To run GSEA with GSEApy, default option
run_GSEA(prerank_file_path=out_dir+prefix+'.rnk', out_dir=out_dir, thread_num=10,
         pathway_file='example/c2.cp.kegg.v2023.1.Hs.symbols.gmt', plot=False,
         min_size=15, max_size=500)

2024-02-01 12:13:27,966 [WARNING] Duplicated values found in preranked stats: 43.59% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2024-02-01 12:13:27,967 [INFO] Parsing data files for GSEA.............................
2024-02-01 12:13:27,979 [INFO] 0010 gene_sets have been filtered out when max_size=500 and min_size=15
2024-02-01 12:13:27,981 [INFO] 0176 gene_sets used for further statistical testing.....
2024-02-01 12:13:27,981 [INFO] Start to run GSEA...Might take a while..................
2024-02-01 12:14:08,780 [INFO] Start to generate gseapy reports, and produce figures...
2024-02-01 12:14:08,781 [INFO] Congratulations. GSEApy runs successfully................



In [10]:
# To run GSEA command line, You need to specify the path to gsea-cli.sh
run_GSEA(prerank_file_path=out_dir+prefix+'.rnk', out_dir=out_dir,
         pathway_file='example/c2.cp.kegg.v2023.1.Hs.symbols.gmt', gsea_out_label=prefix,
         min_size=15, max_size=500, gsea_cli_path='../../GSEA_cmd/gsea-cli.sh', method='cli')

Start running GSEA!
Commandas to run GSEA: ../../GSEA_cmd/gsea-cli.sh GSEAPreranked -gmx example/c2.cp.kegg.v2023.1.Hs.symbols.gmt -collapse No_Collapse -mode Max_probe -norm meandiv -nperm 1000 -rnk example/cond1.vs.cond3.rnk -scoring_scheme weighted -rpt_label cond1.vs.cond3 -create_svgs false -include_only_symbols true -make_sets true -plot_top_x 5 -rnd_seed 42 -set_max 500 -set_min 15 -zip_report false -out example/
Using system JDK.


485      [INFO  ] - Parameters passing to GSEAPreranked.main:
487      [INFO  ] - rnk	example/cond1.vs.cond3.rnk
487      [INFO  ] - gmx	example/c2.cp.kegg.v2023.1.Hs.symbols.gmt
487      [INFO  ] - rpt_label	cond1.vs.cond3
487      [INFO  ] - collapse	No_Collapse
487      [INFO  ] - zip_report	false
487      [INFO  ] - gui	false
487      [INFO  ] - out	example/
487      [INFO  ] - mode	Max_probe
487      [INFO  ] - norm	meandiv
487      [INFO  ] - nperm	1000
487      [INFO  ] - scoring_scheme	weighted
487      [INFO  ] - include_only_symbols	true
487      [INFO  ] - make_sets	true
487      [INFO  ] - plot_top_x	5
487      [INFO  ] - rnd_seed	42
487      [INFO  ] - create_svgs	false
487      [INFO  ] - set_max	500
487      [INFO  ] - set_min	15
524      [INFO  ] - Begun importing: RankedList from: example/cond1.vs.cond3.rnk
952      [INFO  ] - No ranked list collapsing was done .. using original as is
to parse>example/c2.cp.kegg.v2023.1.Hs.symbols.gmt< got: [example/c2.cp.kegg.v2023.1.

GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneSet 101/176 nperm: 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneSet 106/176 nperm: 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneS

In [4]:
# To prune the pathways that have too few or too many genes:
# please keep the min and max gene number setting same as GSEA 
pruned_pathway_dict, gene_universe = prune_gmt(file_name='example/c2.cp.kegg.v2023.1.Hs.symbols.gmt', 
                                        out_file_name=out_dir+'c2.cp.kegg.v2023.1.Hs.symbols.cleaned.gmt', 
                                        expr_matrix_file='example/example_data.txt', 
                                        min_gene_num=15, max_gene_num=500)

53658 genes in the expression matrix
KEGG_GLYCOSPHINGOLIPID_BIOSYNTHESIS_GLOBO_SERIES removed due to gene number
KEGG_NON_HOMOLOGOUS_END_JOINING removed due to gene number
KEGG_CIRCADIAN_RHYTHM_MAMMAL removed due to gene number
KEGG_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTHESIS removed due to gene number
KEGG_TAURINE_AND_HYPOTAURINE_METABOLISM removed due to gene number
KEGG_FOLATE_BIOSYNTHESIS removed due to gene number
KEGG_LIMONENE_AND_PINENE_DEGRADATION removed due to gene number
KEGG_SULFUR_METABOLISM removed due to gene number
KEGG_RIBOSOME removed due to gene number
KEGG_PROTEASOME removed due to gene number


In [21]:
# To perform a pre-processing step (i.e. extracting the top DEGs from DESeq2 results) needed for ORA and Enrichr analyses : 
# By default the following thresholds will be used: pval_threshold=0.05, padj_threshold=0.05, fc_threshold=1, basemean_threshold=2.
# but feel free to adjust the thresholds as needed and keep direction the same as before
# For this toy example, the samples, we have selected a loose threshold of pval_threshold=0.1, padj_threshold=0.1, and fc_threshold=1.5
deg_dict = extract_top_gene(deseq_result_file=out_dir+prefix+'.csv', 
                            num_gene=200, pval_threshold=0.1, padj_threshold=0.1,
                            fc_threshold=1.5, basemean_threshold=2,
                            direction=1)


Based on:
p-value <=  0.1
adjusted p-value <=  0.1
fold change threshold >=  1.5
base Mean >=  2
top N =  200
200 up-regulated DEGs
200 down-regulated DEGs


In [20]:
# To run ORA for up and down-regulated gene lists against the pathway file
run_ora(pathway_dict=pruned_pathway_dict, deg_dict = deg_dict,
        gene_universe_num=len(gene_universe), out_dir=out_dir, out_file_prefix=prefix)


Running ORA for genes in  up
Results written to  example//cond1.vs.cond3.up.ora_result.txt
********************
Running ORA for genes in  down
Results written to  example//cond1.vs.cond3.down.ora_result.txt
********************


In [21]:
# To run Enrichr :
# if the permutation file already exists, it could be directly provided as permutation_file_name to save time
run_enrichr(pathway_dict=pruned_pathway_dict, deg_dict=deg_dict,
            gene_universe=gene_universe, out_file_prefix=prefix, 
            permutation_num=1000, permutation_file_name=out_dir+'enrichr_kegg_permutation_1000.txt',
            out_dir=out_dir)

Running Enrichr for genes in  up
example/enrichr_kegg_permutation_1000.txt does not exist, initializing permutation
Permutation 0
Permutation 50
Permutation 100
Permutation 150
Permutation 200
Permutation 250
Permutation 300
Permutation 350
Permutation 400
Permutation 450
Permutation 500
Permutation 550
Permutation 600
Permutation 650
Permutation 700
Permutation 750
Permutation 800
Permutation 850
Permutation 900
Permutation 950
Permutation result written to  example/enrichr_kegg_permutation_1000.txt
Results written to  example//cond1.vs.cond3.up.enrichr_result.txt
********************
Running Enrichr for genes in  down
example/enrichr_kegg_permutation_1000.txt already exists, reading file now...
Results written to  example//cond1.vs.cond3.down.enrichr_result.txt
********************


**Step 3. Combine the outputs from above methods using PET**


In [6]:
# To run PET when using GSEApy (default):
run_PET(file_prefix=prefix, deg_dict=deg_dict, pathway_dict=pruned_pathway_dict, 
        result_dir=out_dir, gsea_path=out_dir+'gseapy.gene_set.prerank.report.csv', 
        gsea_pos_label='pos', out_dir=out_dir)

GSEA path was provided as file, GSEAPY used.
For genes in  up direction, result files found: 
example/cond1.vs.cond3.up.ora_result.txt 
 example/cond1.vs.cond3.up.enrichr_result.txt 
 example/gseapy.gene_set.prerank.report.up.tmp.csv
Calculating ora rank...
Calculating enrichr rank...
Calculating GSEA rank...
Ensembling results...
Results written to  example//cond1.vs.cond3.up.PET.txt
For genes in  down direction, result files found: 
example/cond1.vs.cond3.down.ora_result.txt 
 example/cond1.vs.cond3.down.enrichr_result.txt 
 example/gseapy.gene_set.prerank.report.down.tmp.csv
Calculating ora rank...
Calculating enrichr rank...
Calculating GSEA rank...
Ensembling results...
Results written to  example//cond1.vs.cond3.down.PET.txt


In [7]:
# Alternatively, To run PET when using GSEA command line:
# you need to specify the gsea_path as the path to the GSEA output directory
run_PET(file_prefix=prefix, deg_dict=deg_dict, pathway_dict=pruned_pathway_dict, 
        result_dir=out_dir, gsea_path='example/cond1.vs.cond3.GseaPreranked.1706807267627/', 
        gsea_pos_label='pos', out_dir=out_dir)

GSEA path was provided as directory, command line version used.
For genes in  up direction, result files found: 
example/cond1.vs.cond3.up.ora_result.txt 
 example/cond1.vs.cond3.up.enrichr_result.txt 
 example/cond1.vs.cond3.GseaPreranked.1706807267627//tmp_result.up.txt
Calculating ora rank...
Calculating enrichr rank...
Calculating GSEA rank...
Ensembling results...
Results written to  example//cond1.vs.cond3.up.PET.txt
For genes in  down direction, result files found: 
example/cond1.vs.cond3.down.ora_result.txt 
 example/cond1.vs.cond3.down.enrichr_result.txt 
 example/cond1.vs.cond3.GseaPreranked.1706807267627//tmp_result.down.txt
Calculating ora rank...
Calculating enrichr rank...
Calculating GSEA rank...
Ensembling results...
Results written to  example//cond1.vs.cond3.down.PET.txt


***
The PET output is a tab-delimited text file. Users may sort the output based on their preferred criteria. 
By default, we have ordered the output based on the **average rank**, which was found to be the most reliable metric according to Benchmark.